In [ ]:
import sys
sys.path.append('../')

In [ ]:
import numpy as np
import open3d as o3d
import time
import logging
from tqdm import tqdm
import pymeshlab
import matplotlib.pyplot as plt

from src.utils import pcd_utils
from src.utils import cityjson_utils
from src.utils import log_utils
from src.utils import plot_utils

from preprocessors.sor import SOR
from preprocessors.spatial_subsample import SpatialSubsample
from modules.floor_split import FloorSplitter
from modules.room_detection import RoomDetector
from modules.primitive_detection import PrimitiveDetector
from modules.room_reconstruct import RoomReconstructor
from modules.mesh_stats import MeshAnalyser


#### Load the interior PointCloud

In [ ]:
in_file = '../datasets/stadskwekerij_subsampled.ply'

In [ ]:
pcd = pcd_utils.read_pointcloud(in_file)
len(pcd.points)

#### 1. Preprocessing

In [ ]:
ss = SpatialSubsample(min_distance=0.03)
sor = SOR(knn=6, n_sigma=2)

preprocessors = [ss, sor]

for obj in preprocessors:
    start = time.time()
    pcd = obj.process(pcd)
    duration = time.time() - start
    print(f'Processor finished in {duration:.2f}s, {len(pcd.points)} points.') 

#### 2. Primitive Detection

In [ ]:
ransac_exe_path = '../cpp_modules/efficient_ransac'
primitive_detector = PrimitiveDetector(ransac_exe_path)

pcd, primitives, primitive_labels = primitive_detector.process(pcd)

In [ ]:
# plot_utils.show_pcd(pcd, primitive_labels)

#### 3. Detect Floors

In [ ]:
floor_splitter = FloorSplitter()
floors = floor_splitter.process(pcd, primitive_labels, primitives)

print(f'Done. Detected {len(floors)} rooms.')

In [ ]:
# plot_utils.show_pcd_floors(pcd, floors)

#### 4. Room Detection

In [ ]:
room_detector = RoomDetector(plot=False)

rooms = []
for floor_mask in floors:
    floor_pcd = pcd.select_by_index(np.where(floor_mask)[0])
    floor_labels = primitive_labels[floor_mask]
    floor_rooms, floor_labels = room_detector.process(floor_pcd, floor_labels)
    primitive_labels[floor_mask] = floor_labels
    for room_i in range(floor_rooms.shape[1]):
        room_mask = np.zeros(len(pcd.points), dtype=bool)
        room_mask[floor_mask] = floor_rooms[:,room_i]
        rooms.append(room_mask)
            
print(f'Done. Detected {len(rooms)} rooms.')

#### 5. Room Reconstruction

In [ ]:
# room_reconstructor = RoomReconstructor()
room_reconstructor = RoomReconstructor('../cpp_modules/polyfit_ransac', '../cpp_modules/polyfit')

room_meshes = []
failed = []
for i in tqdm(np.arange(len(rooms))):
    room_mask = rooms[i]
    room_pts = np.asarray(pcd.points)[room_mask]

    meshset = room_reconstructor.process(room_pts, primitive_labels[room_mask], primitives)
    if meshset is None:
        failed.append(i)
    else:
        room_meshes.append(meshset)

print(f'Failed to reconstruct rooms {failed}')

#### 6. CityJSON Export

In [ ]:
out_path = '../datasets/output.city.json'

cityjson = cityjson_utils.to_cityjson_v1(room_meshes)
cityjson_utils.save_to_file(cityjson, out_path)

#### 7. Room Statistics

In [ ]:
mesh_analyser = MeshAnalyser()  

for i in range(len(room_meshes)):
    room_mesh = room_meshes[i]
    volume, floorarea = mesh_analyser.process(room_mesh)
    try: 
        print(f'Room {i}: {volume:.2f}, {floorarea:.2f}')
    except:
        print(type(volume), type(floorarea))